1. Load and Preprocess Images and Annotations

In [ ]:
#1. Load and Preprocess Images and Annotations
import cv2
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def load_data(data_folder):
    images = []
    labels = []
    for filename in os.listdir(data_folder):
        if filename.endswith(".jpeg"):
            img_path = os.path.join(data_folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                preprocessed_img = preprocess_image(img)
                images.append(preprocessed_img)
                annotation_path = os.path.join(data_folder, filename.replace('.jpeg', '.txt'))
                labels.append(load_annotation(annotation_path))
    return images, labels

def preprocess_image(image):
    # Convert image to RGB (since VGG16 expects 3 channels)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb_image, (128, 128))
    normalized = resized / 255.0
    return normalized

def load_annotation(file_path):
    with open(file_path, 'r') as file:
        content = file.readline().strip().split()
    label = int(content[0])
    return label

data_folder = r'C:\Users\ASUS\OneDrive - techno\Desktop\upskills\data'
images, labels = load_data(data_folder)


2. Create a Dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = np.array(images)
y = np.array(labels)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


3. Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply the transformations to the training data only
datagen.fit(X_train)


4. Model Architecture and Transfer Learning

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load the VGG16 model, excluding the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add new fully connected layers
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


5. Train the Model with Data Augmentation

In [ ]:
# Train the model with data augmentation
batch_size = 32
epochs = 20

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=(X_val, y_val))


6. Fine-Tuning

In [ ]:
# Unfreeze some layers in the base model
import tensorflow as tf
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                         steps_per_epoch=len(X_train) // batch_size,
                         epochs=10,
                         validation_data=(X_val, y_val))


7. Save the Model

In [ ]:
# Save the model to a file
model.save('crop_weed_detection_model.h5')


8. Evaluation and Testing

In [ ]:
# Evaluate the model on the validation set
from tensorflow.keras.models import Model, load_model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {val_acc}")

# Function to make predictions
def detect_crop_weed(image_path):
    image = cv2.imread(image_path)
    preprocessed_image = preprocess_image(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
    prediction = model.predict(preprocessed_image)
    return 'Crop' if prediction[0][0] > 0.5 else 'Weed'

# Load the model from the file
model = load_model('crop_weed_detection_model.h5')

# Test on a new image
test_image_path = os.path.join(data_folder, 'agri_0_456.jpeg')
result = detect_crop_weed(test_image_path)
print(result)


9. Visualize Training History

In [ ]:
# Plot the training and validation loss and accuracy
def plot_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(12, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{title} - Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{title} - Training and Validation Loss')
    plt.show()

plot_history(history, "Initial Training")
plot_history(history_fine, "Fine-Tuning")
